# MLFlow and Seldon

End to end example integrating MLFlow and Seldon, with A/B testing of the models.

## Training

This first section covers how to train models using MLFlow.

### MLproject

The MLproject file defines:
- The environment where the training runs.
- The hyperparameters that can be tweaked. In our case, these are $\{\alpha, l_{1}\}$.
- The interface to train the model.

In [2]:
!ccat ./training/MLproject

name: mlflow-talk

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"


This allows us to have a single command to train the model. 

``` bash
$ mlflow run ./training -P alpha=... -P l1_ratio=...
```

For our example, we will train two versions of the model, which we'll later compare using A/B testing.

- $M_{1}$ with $\alpha = 0.5$
- $M_{2}$ with $\alpha = 0.75$

In [4]:
!mlflow run ./training -P alpha=0.5

2019/11/04 14:27:02 INFO mlflow.projects: === Created directory /tmp/tmp3ynwn0d2 for downloading remote URIs passed to arguments of type 'path' ===
2019/11/04 14:27:02 INFO mlflow.projects: === Running command 'source /home/akash/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-1ecba04797edb7e7f7212d429debd9b664c31651 1>&2 && python train.py 0.5 0.1' in run with ID '7440adcc4c3144aeb9bd234b527565ae' === 
Elasticnet model (alpha=0.500000, l1_ratio=0.100000):
  RMSE: 0.7947931019036529
  MAE: 0.6189130834228138
  R2: 0.18411668718221819
2019/11/04 14:27:03 INFO mlflow.projects: === Run (ID '7440adcc4c3144aeb9bd234b527565ae') succeeded ===


In [3]:
!mlflow run ./training -P alpha=1.0

2019/10/29 22:50:22 INFO mlflow.projects: === Created directory /tmp/tmp2fcxg01g for downloading remote URIs passed to arguments of type 'path' ===
2019/10/29 22:50:22 INFO mlflow.projects: === Running command 'source /home/akash/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-1ecba04797edb7e7f7212d429debd9b664c31651 1>&2 && python train.py 1.0 0.1' in run with ID '539b7e5225844c88a764baaa10112f00' === 
Elasticnet model (alpha=1.000000, l1_ratio=0.100000):
  RMSE: 0.8107373707184711
  MAE: 0.6241295925236751
  R2: 0.15105362812007328
2019/10/29 22:50:24 INFO mlflow.projects: === Run (ID '539b7e5225844c88a764baaa10112f00') succeeded ===


### MLtrack

The `train.py` script uses the `mlflow.log_param()` and `mlflow.log_metric()` commands to track each experiment. These are part of the `MLtrack` API, which tracks experiments parameters and results. These can be stored on a remote server, which can then be shared across the entire team. However, on our example we will store these locally on a `mlruns` folder.

In [5]:
!tree mlruns

mlruns
└── 0
    ├── 7440adcc4c3144aeb9bd234b527565ae
    │   ├── artifacts
    │   │   └── model
    │   │       ├── conda.yaml
    │   │       ├── MLmodel
    │   │       └── model.pkl
    │   ├── meta.yaml
    │   ├── metrics
    │   │   ├── mae
    │   │   ├── r2
    │   │   └── rmse
    │   ├── params
    │   │   ├── alpha
    │   │   └── l1_ratio
    │   └── tags
    │       ├── mlflow.project.backend
    │       ├── mlflow.project.entryPoint
    │       ├── mlflow.project.env
    │       ├── mlflow.source.git.commit
    │       ├── mlflow.source.name
    │       ├── mlflow.source.type
    │       └── mlflow.user
    ├── ae803fa1a775416e82f4b4be2452c70e
    │   ├── artifacts
    │   ├── meta.yaml
    │   ├── metrics
    │   ├── params
    │   │   ├── alpha
    │   │   └── l1_ratio
    │   └── tags
    │       ├── mlflow.project.backend
    │       ├── mlflow.project.entryPoint
    │       ├── mlflow.project.env
    │       ├── mlflow.source.git.commit
    │       ├── mlflow.sourc

We can also run `mlflow ui` to show these visually. This will start the MLflow server in http://localhost:5000.

```bash
$ mlflow ui
```

In [5]:
!mlflow ui

[2019-10-29 22:52:05 +0530] [10249] [INFO] Starting gunicorn 19.9.0
[2019-10-29 22:52:05 +0530] [10249] [INFO] Listening at: http://127.0.0.1:5000 (10249)
[2019-10-29 22:52:05 +0530] [10249] [INFO] Using worker: sync
[2019-10-29 22:52:05 +0530] [10260] [INFO] Booting worker with pid: 10260
^C
[2019-10-29 22:53:00 +0530] [10249] [INFO] Handling signal: int
[2019-10-29 22:53:00 +0530] [10260] [INFO] Worker exiting (pid: 10260)


![MLFlow UI](./images/mlflow-ui.png)

### MLmodel

The `MLmodel` file allows us to version and share models easily. Below we can see an example.

In [7]:
# !ccat ./mlruns/0/ae803fa1a775416e82f4b4be2452c70e/artifacts/model/MLmodel

open ./mlruns/0/ae803fa1a775416e82f4b4be2452c70e/artifacts/model/MLmodel: no such file or directory


As we can see above the `MLmodel` keeps track, between others, of

- The experiment id, `539b7e5225844c88a764baaa10112f00`
- Date 
- Version of `sklearn` 
- How the model was stored

As we shall see shortly, the pre-packaged Seldon's model server will use this file to serve this model.

## Serving

To serve this model we will use Seldon.

### Set up

Before anything, we will first set up the `k8s` cluster.

#### Create k8s cluster

Firstly, we will create a cluster using [kind](https://kind.sigs.k8s.io).

In [8]:
!kind create cluster
!export KUBECONFIG="$(kind get kubeconfig-path --name=kind)"

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.15.3) 🖼
 ✓ Preparing nodes 📦 
 ✓ Creating kubeadm config 📜 
 ✓ Starting control-plane 🕹️ 
 ✓ Installing CNI 🔌 
 ✓ Installing StorageClass 💾 
Cluster creation complete. You can now use the cluster with:

export KUBECONFIG="$(kind get kubeconfig-path --name="kind")"
kubectl cluster-info


We then install Helm and a corresponding service account.

In [3]:
# !kind get clusters
# !export KUBECONFIG="$(kind get kubeconfig-path)"
# !echo $KUBECONFIG
!kubectl cluster-info
!helm init --history-max 200
!kubectl rollout status deploy/tiller-deploy -n kube-system
!kubectl create serviceaccount --namespace kube-system tiller
!kubectl create clusterrolebinding tiller-cluster-rule --clusterrole=cluster-admin --serviceaccount=kube-system:tiller
!kubectl patch deploy --namespace kube-system tiller-deploy -p '{"spec":{"template":{"spec":{"serviceAccount":"tiller"}}}}'

Kubernetes master is running at https://127.0.0.1:34897
KubeDNS is running at https://127.0.0.1:34897/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
$HELM_HOME has been configured at /home/akash/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!
Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out
serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller-cluster-rule created
deployment.extensions/tiller-deploy patched


We can now install `seldon-core` on the new cluster using `helm`.

In [4]:
!helm install \
    seldon-core-operator \
    --name seldon-core \
    --repo https://storage.googleapis.com/seldon-charts \
    --namespace seldon-system \
    --set usagemetrics.enabled=true \
    --set ambassador.enabled=true
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

NAME:   seldon-core
LAST DEPLOYED: Tue Oct 29 23:25:42 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                                        TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.104.152.235  <none>       443/TCP  0s
webhook-server-service                      ClusterIP  10.107.52.253   <none>       443/TCP  0s

==> v1/StatefulSet
NAME                                DESIRED  CURRENT  AGE
seldon-operator-controller-manager  1        1        0s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0  0/1    ContainerCreating  0         0s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     0s

==> v1/ServiceAccount
NAME                              SECRETS  AGE
seldon-core-seldon-core-operator  1        0s

==> v1beta1/CustomResource

Finally, we install `ambassador` which will allow us to reach the Seldon engine in the cluster.

In [5]:
!helm install stable/ambassador --name ambassador --set crds.keep=false
!kubectl rollout status deployment.apps/ambassador

NAME:   ambassador
LAST DEPLOYED: Tue Oct 29 23:26:05 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        1s

==> v1beta1/CustomResourceDefinition
NAME                                          AGE
kubernetesserviceresolvers.getambassador.io   1s
authservices.getambassador.io                 1s
tracingservices.getambassador.io              1s
tlscontexts.getambassador.io                  1s
modules.getambassador.io                      1s
ratelimitservices.getambassador.io            1s
tcpmappings.getambassador.io                  1s
kubernetesendpointresolvers.getambassador.io  1s
mappings.getambassador.io                     1s
consulresolvers.getambassador.io              0s

==> v1beta1/ClusterRole
ambassador  0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  0s

==> v1/Service
NAME               TYPE          CLUSTER-IP      EXTERNAL-IP  PORT(S)                     AGE
ambassador-admins  ClusterIP   

#### Forward port

Once the cluster has been created, we need to allow access from the outside to the `ambassador` gateway.
One way to do this is to use the `kubectl port-forward` command.
In particular, we will forward port `8003` of our local host to the cluster's gateway.

This command needs to run constantly on the background, so **please make sure you run it on a separate terminal**.

```bash
kubectl \
    port-forward \
    $(kubectl get pods \
        -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') \
    8003:8080
```

#### Install Seldon Core Analytics

Later, after we deploy the models, we will compare their performance using Seldon Core's integration with Prometheus and Grafana.
For that part to work, we first need to install Grafana.

In [6]:
!helm install seldon-core-analytics --name seldon-core-analytics \
     --repo https://storage.googleapis.com/seldon-charts \
     --set grafana_prom_admin_password=password \
     --set persistence.enabled=false

NAME:   seldon-core-analytics
LAST DEPLOYED: Tue Oct 29 23:27:40 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           1s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          1s
grafana-prom-deployment  1        1        1           0          1s
prometheus-deployment    1        1        1           0          1s

==> v1beta1/ClusterRole
NAME        AGE
prometheus  1s

==> v1beta1/ClusterRoleBinding
NAME        AGE
prometheus  1s

==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.110.189.250  <none>       80/TCP        1s
grafana-prom              NodePort   10.100.241.163  <none>       80:31194/TCP  1s
prometheus-node-exporter  ClusterIP  None            <none>       9100/T

To access Grafana, it will be necessary to forward the port to the respective pod as we did previously to access the Seldon Core deployment.
The credentials will be simply `admin` // `password`.

This command needs to run constantly on the background, so **please make sure you run it on a separate terminal**.

```bash
$ kubectl port-forward \
    $(kubectl get pods \
        -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') \
    3000:3000
```

### Deploy models

Once the cluster is set up, the next step will to upload these models into a common repository and to deploy two `SeldonDeployment` specs to `k8s`.

#### Upload models (optional)

To make sure our `k8s` pods have access to the models we have just trained using `MLflow`, we will upload them into Google Cloud Storage. Note that to run these commands you need write access into the `gs://seldon-models` bucket and you need to have `gsutil` set up.

We will upload both versions of the model to:

- `gs://seldon-models/mlflow/model-a`
- `gs://seldon-models/mlflow/model-b`

In [2]:
# !gsutil cp -r mlruns/0/169119a7fe1e4b31a746e891499552b0/artifacts/model gs://seldon-models/mlflow/model-a
# !gsutil cp -r mlruns/0/5a6be5a1ef844783a50a6577745dbdc3/artifacts/model gs://seldon-models/mlflow/model-b

#### Deploy specs

We will deploy our A/B inference graph to our `k8s` cluster. As we can see below, we will route 50% of the traffic to each of the models.

In [7]:
!pygmentize ./serving/model-a-b.yaml
!kubectl apply -f ./serving/model-a-b.yaml 

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: wines-classifier
spec:
  name: wines-classifier
  predictors:
  - graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/model-a
      name: wines-classifier
    name: model-a
    replicas: 1
    traffic: 50
  - graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/model-b
      name: wines-classifier
    name: model-b
    replicas: 1
    traffic: 50
seldondeployment.machinelearning.seldon.io/wines-classifier configured


We can verify these have been deployed by checking the pods and `SeldonDeployment` resources in the cluster.

In [8]:
!kubectl get pods

NAME                                                READY   STATUS      RESTARTS   AGE
alertmanager-deployment-db58649dd-xt5fm             1/1     Running     0          77m
ambassador-5c76696fcc-9wvj4                         1/1     Running     0          5h23m
ambassador-5c76696fcc-bb96c                         1/1     Running     0          5h23m
ambassador-5c76696fcc-mxv8k                         1/1     Running     0          5h23m
grafana-prom-deployment-8564b575dd-plchb            1/1     Running     0          77m
grafana-prom-import-dashboards-jvnq9                0/1     Completed   0          77m
prometheus-deployment-d57b5c748-gr5mf               1/1     Running     0          77m
prometheus-node-exporter-c5rdn                      1/1     Running     0          77m
wines-classifier-model-a-77efeb1-766f5c8458-548m6   0/2     Running     0          29s
wines-classifier-model-a-77efeb1-7c495775cb-dhbq5   2/2     Running     0          74m
wines-classifier-model-b-77efeb1-78cc

In [9]:
!kubectl get sdep

NAME               AGE
wines-classifier   74m


#### Test models

We will now run a sample query to test that the inference graph is working.

In [10]:
!http \
    --print b \
    localhost:8003/seldon/default/wines-classifier/api/v0.1/predictions \
    data:='{\
        "names": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], \
        "ndarray": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]] \
    }'

{
    "data": {
        "names": [],
        "ndarray": [
            5.550530190667395
        ]
    },
    "meta": {
        "metrics": [],
        "puid": "ja58a82rrf2amp44rtsl4af7gp",
        "requestPath": {
            "wines-classifier": "seldonio/mlflowserver_rest:0.2"
        },
        "routing": {},
        "tags": {}
    }
}



### Analytics

Now that we have both models running in production, we can analyse their performance using Seldon Core's integration with Prometheus and Grafana.
To do so, we will iterate over the training set (which can be foud in `./training/wine-quality.csv`), making a request and sending the feedback of the prediction.

Since the `/feedback` endpoint requires a `reward` signal (i.e. higher better), we will simulate one as

$$
  R(x_{n})
    = \begin{cases}
        \frac{1}{(y_{n} - f(x_{n}))^{2}} &, y_{n} \neq f(x_{n}) \\
        500 &, y_{n} = f(x_{n})
      \end{cases}
$$

, where $R(x_{n})$ is the reward for input point $x_{n}$, $f(x_{n})$ is our trained model and $y_{n}$ is the actual value.

In [8]:
import pandas as pd
import numpy as np
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    gateway="ambassador", 
    namespace="default",
    deployment_name='wines-classifier')

df = pd.read_csv("./training/wine-quality.csv")

def _get_reward(y, y_pred):
    if y == y_pred:
        return 500    
    
    return 1 / np.square(y - y_pred)

def _test_row(row):
    input_features = row[:-1]
    feature_names = input_features.index.to_list()
    X = input_features.values.reshape(1, -1)
    y = row[-1].reshape(1, -1)
    
    r = sc.predict(
        data=X,
        names=feature_names)
    
    y_pred = r.response.data.tensor.values
    reward = _get_reward(y, y_pred)
    sc.feedback(
        prediction_request=r.request,
        prediction_response=r.response,
        reward=reward)
    
    return reward[0]

df.apply(_test_row, axis=1)

ModuleNotFoundError: No module named 'seldon_core'

 We can now access the Grafana dashboard in http://localhost:3000 (credentials are `admin` // `password`). Inside the portal, we will go to the Prediction Analytics dashboard.
 
 
We can see a snapshot below.

![Seldon Analytics](./images/seldon-analytics.png)